In [31]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("../RE instacart/datasets/df_order_products__prior_sample.csv")[:50000]

In [18]:
df.head()

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered
0,61,8,23423,1,1
1,333,40,10070,1,1
2,334,40,42450,2,1
3,335,40,33198,3,1
4,336,40,34866,4,1


In [21]:
products = []
for i in df["order_id"].unique():
    products.append(list(df[df.order_id == i]['product_id']))

In [22]:
len(products)

5003

In [23]:
df_products = pd.read_csv("../RE instacart/datasets/products.csv")
prod_map = dict(zip(df_products["product_id"],df_products["product_name"]))
def getProductName(product_id):   
    return prod_map.get(product_id,"MissingId")

In [24]:
items = [ [getProductName(prod_id) for prod_id in bucket] for bucket in produts]
#for bucket in products:

In [25]:
items[:5]

[['Original Hawaiian Sweet Rolls'],
 ['Organic 1% Low Fat Milk',
  'Macaroni & Cheese',
  'Sparkling Natural Mineral Water',
  'Chocolate Milk 1% Milkfat'],
 ['Caramel Vanilla Cream Light Roast K-Cup Packs Arabica Coffee',
  'Half And Half Ultra Pasteurized',
  'Purified Water',
  'Ultra Soft & Strong® Toilet Paper Double Rolls',
  'Smoothies, Strawberries Wild'],
 ['Bag of Organic Bananas',
  'Plain Greek Yogurt',
  'Maple Glazed Honey Ham',
  'Organic Grade A Large Brown Eggs',
  'Cherry Garcia Ice Cream',
  'Prosciutto',
  'Thick & Crispy Tortilla Chips',
  'High Pulp Orange Juice'],
 ['Organic Ginger Root',
  'Organic Chicken Bone Broth',
  'Organic Low Sodium Chicken Cooking Stock',
  'Original Soy Creamer',
  'Liquid Egg Whites',
  'Americone Dream® Ice Cream',
  'Rosemary Ham',
  'Organic Mayonnaise',
  'Organic Sunday Bacon',
  'Organic Beefsteak Tomato',
  'Organic Roma Tomato']]

In [26]:
import pickle
with open('Instadata.txt', 'wb') as f:
    pickle.dump(items, f)

In [27]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [28]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [29]:
wordlist.append('UNKNOWN_TOKEN')

In [32]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [33]:
len(wordlist), len(embeddings)

(11207, 11207)

In [34]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [35]:
word_to_index['UNKNOWN_TOKEN']

11206

In [48]:
basket = items.copy()

In [49]:
unknown_token = 'UNKNOWN_TOKEN'

In [51]:
for i, sent in enumerate(basket):
    basket[i] = [w if w in word_to_index else unknown_token for w in sent]

In [53]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in basket])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in basket])

In [54]:
x_example, y_example = X_train[10], y_train[10]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
Organic Half & Half
[33]

y:
Chunky Peanut Butter
[2866]


In [55]:
len(X_train), X_train.shape

(5003, (5003,))

In [56]:
X_train[:10]

array([[], [288, 157, 68], [4900, 932, 252, 4089],
       [1, 267, 3393, 203, 1525, 1377, 475],
       [57, 419, 578, 1118, 999, 1095, 1808, 591, 281, 3859],
       [10, 511, 4067, 9052, 8288, 6685, 8381, 4998],
       [1203, 812, 2706, 2645, 3163],
       [79, 2401, 7, 219, 25, 7543, 1160, 596, 8585], [298, 280, 54],
       [16, 793, 199, 2513, 207, 58, 555, 12, 804]], dtype=object)

In [57]:
y_train[:10]

array([[], [157, 68, 1945], [932, 252, 4089, 5146],
       [267, 3393, 203, 1525, 1377, 475, 2026],
       [419, 578, 1118, 999, 1095, 1808, 591, 281, 3859, 148],
       [511, 4067, 9052, 8288, 6685, 8381, 4998, 923],
       [812, 2706, 2645, 3163, 529],
       [2401, 7, 219, 25, 7543, 1160, 596, 8585, 2609], [280, 54, 140],
       [793, 199, 2513, 207, 58, 555, 12, 804, 39]], dtype=object)

In [58]:
X_train = np.array([var for var in X_train if var])
y_train = np.array([var for var in y_train if var])

In [68]:
import tensorflow as tf
import numpy as np
x = X_train
y = y_train
max_length =  10
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [word_to_index['UNKNOWN_TOKEN']] * (max_length - len(row)))
    else :
        x_padded.append(row[:10])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [word_to_index['UNKNOWN_TOKEN']] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:10])

x_padded = np.array(x_padded)
y_padded = np.array(y_padded)

In [69]:
x_padded.shape, y_padded.shape

((4749, 10), (4749, 10))

In [70]:
xtrain, ytrain = x_padded[:4000], y_padded[:4000]
xtest, ytest = x_padded[4000:], y_padded[4000:]

In [71]:
tf.reset_default_graph()
from tensorflow.contrib import rnn 
hm_epochs = 150
batch_size = 500
num_steps = 10
state_size = 200
vocab_size = len(wordlist)

# Placeholders
x = tf.placeholder(tf.int32, [None, num_steps]) # [batch_size, num_steps]
seqlen = tf.placeholder(tf.int32, [None])
y = tf.placeholder(tf.int32, [None, num_steps])

def train_neural_network(x): 
    
        def next_batch(step):
            p,q = xtrain[batch_size*step:batch_size*(step+1)], ytrain[batch_size*step:batch_size*(step+1)]
            return p,q
        
        def test_batch(stp):    
            a,b = xtest[batch_size*stp:batch_size*(stp+1)], ytest[batch_size*stp:batch_size*(stp+1)]
            return a,b
              
        def lstm_neural_network(x):
            # Embedding layer
            rnn_inputs = tf.nn.embedding_lookup(embeddings, x)
            rnn_inputs = tf.cast(rnn_inputs, dtype=tf.float32)
            print(rnn_inputs)
            
            # RNN
            inputs = tf.unstack(rnn_inputs, num=num_steps, axis=1)   
            cell = tf.contrib.rnn.GRUCell(state_size)
            cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5)
            print("this is rnn going in:", rnn_inputs)
            rnn_outputs, rnn_states = tf.contrib.rnn.static_rnn(cell, inputs, dtype=tf.float32)

            rnn_output = tf.reshape(tf.concat(axis=1, values=rnn_outputs), [-1, state_size])
            print("this is the output:",rnn_output)
            softmax_w = tf.get_variable("softmax_w", [state_size, vocab_size], dtype=tf.float32)
            softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
            logits = tf.add(tf.matmul(rnn_output, softmax_w),softmax_b, name = "logits")
            return logits

        prediction = lstm_neural_network(x)
        loss = tf.argmax(prediction, axis=1)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [prediction],
            [tf.reshape(y[:batch_size], [-1])],
            [tf.ones([batch_size * num_steps], dtype=tf.float32)])
        cost = tf.reduce_sum(loss) / batch_size
        optimizer = tf.train.AdamOptimizer().minimize(cost)   
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())
            for epoch in range(hm_epochs):
                epoch_loss = 0 
                for step in range(int(len(xtrain)/batch_size)):
                    epoch_x, epoch_y = next_batch(step)
                    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                    epoch_loss += c
                print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "./model")
            print("Model saved in file: %s" % save_path)
            
            for stp in range(int(len(xtest)/batch_size)):
                s,u = test_batch(stp)
                correct = tf.equal((tf.argmax(prediction,1)),tf.cast(tf.reshape(tf.concat(axis=1, values= u), [-1]),tf.int64))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                print( 'Batch #', stp, 'Accuracy by each batch:',accuracy.eval({x: s, y: u}))

In [72]:
train_neural_network(x)

Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is rnn going in: Tensor("Cast:0", shape=(?, 10, 200), dtype=float32)
this is the output: Tensor("Reshape:0", shape=(?, 200), dtype=float32)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 150 loss: 743.870384216
Epoch 1 completed out of 150 loss: 719.909934998
Epoch 2 completed out of 150 loss: 627.529769897
Epoch 3 completed out of 150 loss: 536.962463379
Epoch 4 completed out of 150 loss: 514.900238037
Epoch 5 completed out of 150 loss: 509.803375244
Epoch 6 completed out of 150 loss: 504.931190491
Epoch 7 completed out of 150 loss: 502.242801666
Epoch 8 completed out of 150 loss: 500.407283783
Epoch 9 completed out of 150 loss: 498.887355804
Epoch 10 completed out of 150 loss: 497.759822845
Epoch 11 completed out of 150 loss: 496.418159485
Epoch 12 completed out of 150 loss: 495.040977478
Epoch 13 completed out of 150 loss: 493.514068604
Epoch 14 completed out of 150 loss: 491

In [73]:
len(x_padded)

4749

In [76]:
tt = np.array(x_padded[4649:4749])

In [138]:
test_prod = [["California White Jasmine Rice","Squares Dark 60% Cacao Chocolate"]]

In [146]:
test_prod_embeds = [[word_to_index[prod] for prod in bucket] for bucket in test_prod ]

In [147]:
test_prod_embeds

[[3157, 9009]]

In [157]:
test_x_padded = []
for row in test_prod_embeds:    
        print(row)
        if len(row) <= max_length:
            test_x_padded.append(row + [word_to_index['UNKNOWN_TOKEN']] * (max_length - len(row)))
        else :
            test_x_padded.append(test_prod_embeds[:10])

[3157, 9009]


In [158]:
test_x_padded

[[3157, 9009, 11206, 11206, 11206, 11206, 11206, 11206, 11206, 11206]]

In [159]:
len(test_x_padded)

1

In [170]:
sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('model.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
sess.run(tf.global_variables_initializer())
graph = tf.get_default_graph()

#context = graph.get_tensor_by_name("context:0") 
#utterance = graph.get_tensor_by_name("utterance:0")
#target = graph.get_tensor_by_name("target:0")

op_to_restore = graph.get_tensor_by_name("logits:0")
#qe,an,lbl = test_batch(0)
feed_dict={x: test_x_padded}
#feed_dict={x: tt}
results = sess.run(op_to_restore,feed_dict)
print("These are results:", results)
results = np.reshape(results, [len(test_x_padded), num_steps, vocab_size])

INFO:tensorflow:Restoring parameters from ./model
These are results: [[ 0.00732417  0.0321641   0.05668192 ..., -0.00361386 -0.04422812
  -0.02499113]
 [-0.01966417 -0.00751682  0.05242439 ...,  0.02392079 -0.0143792
   0.0313725 ]
 [-0.02138201 -0.01870059  0.01916721 ..., -0.00908546 -0.06976476
   0.00417476]
 ..., 
 [-0.00337588 -0.01292137  0.02924705 ..., -0.01917645 -0.0014609
  -0.00765509]
 [-0.00208684 -0.00583546  0.01550917 ..., -0.00766807  0.00121961
  -0.01017604]
 [ 0.01157808 -0.01246053  0.01432914 ...,  0.0015014   0.01691655
  -0.0150364 ]]


In [171]:
r1= results.argmax(axis=2)
#r2=tt[:20]

In [172]:
(r1).shape

(1, 10)

In [173]:
r1

array([[  845,  2477,  8145,  2796,  8625, 10237,  1354,  8263, 10051,
         7531]], dtype=int64)

In [174]:
for pre in (r1[:10]):
    print("Starts")
    print("\ny': These are the predictions made \n%s" % (wordlist[pre[-1]] ))
    #print("\ny': These are the predictions made \n%s" % (",".join([wordlist[x] for x in pre])))
    #print("\ny These are the ground truths :\n%s" % (",".join([wordlist[x] for x in the])))
    print("End\n")

Starts

y': These are the predictions made 
Salmon Pinwheels
End

